In [ ]:
# Установка необходимых пакетов
!pip install vllm transformers datasets peft torch accelerate
!pip install bitsandbytes  # для 4-bit/8-bit квантования

# Импорт библиотек
from vllm import LLM, SamplingParams
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, 
    TrainingArguments, Trainer, DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset
import torch
import json

In [ ]:
# Инициализация модели для инференса
model_name = "mistralai/Mistral-7B-Instruct-v0.2"  # пример модели

# Параметры инициализации vLLM
llm = LLM(
    model=model_name,
    tensor_parallel_size=1,           # Количество GPU для тензорного параллелизма
    gpu_memory_utilization=0.9,       # Использование памяти GPU (0.0-1.0)
    max_model_len=4096,               # Максимальная длина контекста
    quantization="awq",               # Квантование: "awq", "squeezellm", None
    trust_remote_code=True,           # Доверять пользовательскому коду
    enforce_eager=True,               # Для отладки - отключает оптимизации
)

# Параметры семплирования
sampling_params = SamplingParams(
    temperature=0.7,                  # Случайность (0.0-1.0)
    top_p=0.9,                        # Nucleus sampling
    top_k=50,                         # Top-k sampling
    max_tokens=512,                   # Максимальная длина генерируемого текста
    presence_penalty=0.0,             # Штраф за повторяющиеся токены
    frequency_penalty=0.0,            # Штраф за частые токены
    stop=["</s>", "\n\n"],            # Стоп-токены
    n=1,                              # Количество вариантов генерации
    best_of=3,                        # Лучший из n вариантов
)

# Подготовка промптов
prompts = [
    "Объясни концепцию машинного обучения:",
    "Напиши код для сортировки пузырьком на Python:",
    "Какие основные преимущества трансформеров в NLP?"
]

# Запуск инференса
outputs = llm.generate(prompts, sampling_params)

# Обработка результатов
for i, output in enumerate(outputs):
    prompt = prompts[i]
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt}")
    print(f"Generated: {generated_text}")
    print(f"Tokens generated: {len(output.outputs[0].token_ids)}")
    print("-" * 50)

In [ ]:
# Потоковая генерация для длинных текстов
def stream_generation(prompts, sampling_params):
    for output in llm.generate(prompts, sampling_params, use_tqdm=True):
        # Реализация потоковой обработки
        if hasattr(output, 'outputs') and output.outputs:
            yield output.outputs[0].text

# Пример использования потоковой обработки
streaming_prompts = ["Расскажи подробно о глубоком обучении:"]
for result in stream_generation(streaming_prompts, sampling_params):
    print("Получен фрагмент:", result[:100] + "...")

In [ ]:
# Формат данных для instruction tuning
def prepare_instruction_data():
    # Пример данных - замените на свои
    instructions = [
        {
            "instruction": "Объясни концепцию",
            "input": "нейронные сети",
            "output": "Нейронные сети - это вычислительные системы..."
        },
        {
            "instruction": "Напиши код", 
            "input": "функция сложения на Python",
            "output": "def add(a, b):\n    return a + b"
        }
    ]
    
    formatted_data = []
    for item in instructions:
        # Форматирование в чат-формат
        text = f"<s>[INST] {item['instruction']}: {item['input']} [/INST] {item['output']} </s>"
        formatted_data.append({"text": text})
    
    return Dataset.from_list(formatted_data)

# Создание dataset
dataset = prepare_instruction_data()
train_dataset = dataset.train_test_split(test_size=0.1)['train']
eval_dataset = dataset.train_test_split(test_size=0.1)['test']

In [ ]:
# Параметры LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,      # Тип задачи - causal language modeling
    inference_mode=False,              # Режим обучения
    r=16,                              # Rank матриц LoRA
    lora_alpha=32,                     # Коэффициент масштабирования
    lora_dropout=0.1,                  # Dropout для LoRA
    target_modules=[
        "q_proj",      # Проекции запросов
        "k_proj",      # Проекции ключей  
        "v_proj",      # Проекции значений
        "o_proj",      # Выходные проекции
        "gate_proj",   # Gate проекции (для Mistral)
        "up_proj",     # Up проекции
        "down_proj",   # Down проекции
    ],
    bias="none",                       # Обработка bias: "none", "all", "lora_only"
)

# Загрузка базовой модели
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,        # Тип данных
    device_map="auto",                 # Автоматическое распределение по GPU
    trust_remote_code=True,
    load_in_4bit=True,                 # 4-bit квантование для экономии памяти
)

# Применение LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()     # Покажет количество обучаемых параметров

In [ ]:
# Параметры обучения
training_args = TrainingArguments(
    output_dir="./lora-finetuned-model",  # Директория для сохранения
    per_device_train_batch_size=4,        # Batch size на устройство
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,        # Накопление градиентов
    learning_rate=2e-4,                   # Скорость обучения для LoRA
    num_train_epochs=3,                   # Количество эпох
    logging_steps=10,                     # Шаг логирования
    eval_steps=100,                       # Шаг оценки
    save_steps=500,                       # Шаг сохранения
    warmup_steps=100,                     # Шаги прогрева
    evaluation_strategy="steps",          # Стратегия оценки
    save_strategy="steps",                # Стратегия сохранения
    load_best_model_at_end=True,          # Загрузка лучшей модели
    metric_for_best_model="eval_loss",    # Метрика для лучшей модели
    greater_is_better=False,
    fp16=True,                            # Использование mixed precision
    report_to="none",                     # Отключение внешних логгеров
    ddp_find_unused_parameters=False,
)

# Data collator для language modeling
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Установка pad токена

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Не использовать masked language modeling
)

In [ ]:
# Создание Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Запуск обучения
trainer.train()

# Сохранение модели
trainer.save_model()
tokenizer.save_pretrained("./lora-finetuned-model")

### peft inference

In [ ]:
from peft import PeftModel

# Загрузка базовой модели
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# Загрузка адаптеров LoRA
model = PeftModel.from_pretrained(
    base_model,
    "./lora-finetuned-model",
    torch_dtype=torch.bfloat16,
)

# Переход в режим инференса
model.eval()

# Инференс с адаптированной моделью
def generate_with_lora(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Тестирование
test_prompt = "Объясни принцип работы внимания в трансформерах:"
result = generate_with_lora(test_prompt)
print(result)

### слияние весов и vllm инференс

In [ ]:
# Слияние адаптеров LoRA с базовой моделью
model = PeftModel.from_pretrained(
    base_model,
    "./lora-finetuned-model",
    torch_dtype=torch.bfloat16,
)

# Слияние весов
merged_model = model.merge_and_unload()

# Сохранение объединенной модели
merged_model.save_pretrained("./merged-model")
tokenizer.save_pretrained("./merged-model")

# Загрузка в vLLM для быстрого инференса
merged_llm = LLM(
    model="./merged-model",
    tensor_parallel_size=1,
    gpu_memory_utilization=0.8,
    max_model_len=4096,
)

# Инференс с объединенной моделью
prompts = ["Объясни трансформеры простыми словами:"]
sampling_params = SamplingParams(temperature=0.7, max_tokens=256)
outputs = merged_llm.generate(prompts, sampling_params)

for output in outputs:
    print(output.outputs[0].text)